In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
from sklearn.svm import SVC, LinearSVC

#vectorizer = SentenceTransformer('intfloat/multilingual-e5-large-instruct', cache_folder="N:\AI\Transformers_cache")
vectorizer = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
#vectorizer = SentenceTransformer("Salesforce/SFR-Embedding-Mistral")
#model = MLPClassifier(random_state=42, alpha=0.25)
model = LinearSVC(C=0.5, random_state=42)

data = pd.read_csv('Annotated_data.csv')
distortion_data = data[data['Dominant Distortion'] != 'No Distortion']
distortion_data = distortion_data.reset_index(drop=True)

discreteData = data[['Id_Number', 'Patient Question', 'Dominant Distortion']].copy()
discreteData.loc[discreteData['Dominant Distortion'] != 'No Distortion', 'Dominant Distortion'] = 'Distorted'

X_train, X_test, y_train, y_test = train_test_split(discreteData['Patient Question'], discreteData['Dominant Distortion'], test_size=0.2, random_state=42)

# print the classes in discreteData
print(discreteData['Dominant Distortion'].value_counts())

c:\Users\meaps\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Distorted        1597
No Distortion     933
Name: Dominant Distortion, dtype: int64


In [3]:
# vectorize the training data
X_train_vector = vectorizer.encode(X_train.tolist())
X_test_vector = vectorizer.encode(X_test.tolist())

# store the vectors in a dataframe
X_train_vector = pd.DataFrame(X_train_vector)
X_test_vector = pd.DataFrame(X_test_vector)

# store the dataframe in a csv file
X_train_vector.to_csv('X_train_vector.csv', index=False)
X_test_vector.to_csv('X_test_vector.csv', index=False)

# store the y_train and y_test in a csv file
y_train.to_csv('y_train.csv', index=False)
y_test.to_csv('y_test.csv', index=False)



In [4]:
# read the vectors from the csv file
X_train_vector = pd.read_csv('X_train_vector.csv')
X_test_vector = pd.read_csv('X_test_vector.csv')

# read the y_train and y_test from the csv file
y_train = pd.read_csv('y_train.csv')
y_test = pd.read_csv('y_test.csv')

# train the model
model.fit(X_train_vector, y_train)

# predict the test data
y_pred = model.predict(X_test_vector)

# calculate the f1 score
f1 = f1_score(y_test, y_pred, average='weighted')
print(f1)

0.7134102938155855


c:\Users\meaps\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\meaps\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
